# Library

In [31]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [32]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [33]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [34]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

# 페이지 접속

In [35]:
# 페이지 주소
while True:
    try:
        section_number = int(input('뉴스의 섹션 번호를 입력하세요(1:정치, 2:경제, 3:사회, 4:생활/문화, 5:IT/과학, 6:세계): '))
    except:
        continue
    if section_number == 1:
        url = 'https://news.naver.com/section/100' # 네이버 뉴스 > 정치
        print('정치 섹션의 뉴스를 크롤링합니다.')
        break
    elif section_number == 2:
        url = 'https://news.naver.com/section/101' # 네이버 뉴스 > 경제
        print('경제 섹션의 뉴스를 크롤링합니다.')
        break
    elif section_number == 3:
        url = 'https://news.naver.com/section/102' # 네이버 뉴스 > 사회
        print('사회 섹션의 뉴스를 크롤링합니다.')
        break
    elif section_number == 4:
        url = 'https://news.naver.com/section/103' # 네이버 뉴스 > 생활/문화
        print('생활/문화 섹션의 뉴스를 크롤링합니다.')
        break
    elif section_number == 5:
        url = 'https://news.naver.com/section/105' # 네이버 뉴스 > IT/과학
        print('IT/과학 섹션의 뉴스를 크롤링합니다.')
        break
    elif section_number == 6:
        url = 'https://news.naver.com/section/104' # 네이버 뉴스 > 세계
        print('세계 섹션의 뉴스를 크롤링합니다.')
        break

세계 섹션의 뉴스를 크롤링합니다.


## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [36]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

## selenium 작동

In [37]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [49]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

## 헤드라인 뉴스 섹션 로드

In [50]:
# 헤드라인 뉴스
headline_news_more_view_button_xpath = '//*[@id="newsct"]/div[1]/div[2]/a' # 헤드라인 더 보기 버튼

# 헤드라인 더 보기 버튼 로딩 대기
headline_news_more_view_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, headline_news_more_view_button_xpath))
)

# 헤드라인 더보기 버튼 클릭
headline_news_more_view_button.click()
driver.implicitly_wait(10)

# 헤드라인 뉴스 수집

In [51]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'세계 : 네이버 뉴스'

In [52]:
# 헤드라인 뉴스 섹션
headline_news_section = soup.select_one('ul.sa_list')
len(headline_news_section)

21

In [53]:
# 뉴스 타이들 리스트
headline_news_list = headline_news_section.select('li.sa_item._SECTION_HEADLINE')
print('헤드라인 뉴스 개수 :', len(headline_news_list))

헤드라인 뉴스 개수 : 10


# 수집한 데이터 전처리

In [ ]:
link_list = []
title_list = []
content_list = []
press_list = []
related_news_count_list = []

for i in range(len(headline_news_list)):
    # 기사 내용 섹션
    news_content_section = headline_news_list[i].select_one('div.sa_text')

    # 기사 링크
    try:
        link = news_content_section.select_one('a.sa_text_title._NLOG_IMPRESSION')['href']
    except:
        link = np.nan
        print(i, '기사 링크 없음')
    
    # 타이틀
    try:
        title = news_content_section.select_one('strong.sa_text_strong').text.strip()
    except:
        title = np.nan
        print(i, '타이틀 없음')
    
    # 기사 내용
    try:
        content = news_content_section.select_one('div.sa_text_lede').text.strip()
    except:
        content = np.nan
        print(i, '기사 내용 없음')
    
    ## 언론사
    try:
        press = news_content_section.select_one('div.sa_text_press').text
    except:
        press = np.nan
        print(i, '언론사 없음')
    
    ## 관련 뉴스 개수
    try:
        related_news_count = int(news_content_section.select_one('span.sa_text_cluster_num').text)
    except:
        related_news_count = np.nan
        print(i, '관련 뉴스 개수 없음')
    
    # 정보 리스트에 담기
    link_list.append(link)
    title_list.append(title)
    content_list.append(content)
    press_list.append(press)
    related_news_count_list.append(related_news_count)

In [55]:
len(link_list), len(title_list), len(content_list), len(press_list), len(related_news_count_list)

(10, 10, 10, 10, 10)

In [56]:
data_dict = {
    '번호': range(1, len(link_list)+1),
    '기사 링크': link_list,
    '기사 제목': title_list,
    '기사 내용': content_list,
    '언론사': press_list,
    '관련 뉴스 개수': related_news_count_list
    }
df = pd.DataFrame(data_dict)
df

,번호,기사 링크,기사 제목,기사 내용,언론사,관련 뉴스 개수
0,1,https://n.news.naver.com/mnews/article/056/001...,한미 “계엄사태로 연기됐던 주요 외교·안보 일정 완전 재개”,"한미 양국이 한국의 계엄사태로 연기됐던 양국간 주요 외교, 안보 일정을 완전히 재개...",KBS,59
1,2,https://n.news.naver.com/mnews/article/055/000...,일본제철의 US스틸 인수 '불허' 바이든 결정만 남아,일본제철의 US스틸 인수를 심사해온 미국 재무부 산하 외국인투자심의위원회(CFIUS...,SBS,6
2,3,https://n.news.naver.com/mnews/article/056/001...,젤렌스키 “러시아 쿠르스크에서 북한군 사상자 3천명 넘어”,볼로디미르 젤렌스키 우크라이나 대통령은 북한군이 러시아 서부 쿠르스크 지역에서 우크...,KBS,33
3,4,https://n.news.naver.com/mnews/article/214/000...,"러 ""내년 전승절 붉은광장 열병식에 북한군 참가할 수도""","내년 러시아의 제2차 세계대전 승리 기념일, 전승절 행사에 북한군이 참여할 수 있다...",MBC,32
4,5,https://n.news.naver.com/mnews/article/003/001...,"日증시, '닛산 합병' 혼다 10%대 급등…닛케이지수 0.12%↑",24일 도쿄 주식시장에서 닛케이225지수(닛케이평균주가)는 상승세로 출발했다. 이날...,뉴시스,65
5,6,https://n.news.naver.com/mnews/article/001/001...,"""우크라 국민 73% 핵무장 복원 지지""",신창용 특파원 = 우크라이나 국민 4명 중 3명은 핵무장 복원에 찬성하는 것으로 나...,연합뉴스,6
6,7,https://n.news.naver.com/mnews/article/277/000...,"美의회조사국 ""韓 탄핵 정국에 尹 외교정책 지속성 의문""",미 의회조사국(CRS)이 한국의 계엄과 탄핵 사태로 한·미·일 3자 협력 등 윤석열...,아시아경제,20
7,8,https://n.news.naver.com/mnews/article/215/000...,"""빠지면 더 산다""…비트코인 7주 연속 매입한 '이곳'",비트코인 가격 하락세가 지속되고 있는 가운데 세계 최대 암호화폐 기업 마이크로스트래...,한국경제TV,60
8,9,https://n.news.naver.com/mnews/article/052/000...,"트럼프, 덴마크 대사 지명...또 '그린란드 편입' 주장",도널드 트럼프 미국 대통령 당선인이 덴마크 주재 미국 대사 지명을 발표하면서 덴마크...,YTN,22
9,10,https://n.news.naver.com/mnews/article/052/000...,"""이시바, 트럼프 취임 전 회동 검토...고용 공헌 강조할 듯""",이시바 시게루 일본 총리가 다음 달 중순 미국을 방문해 트럼프 대통령 당선인과 취임...,YTN,5


# 데이터 출력

In [25]:
current_date = time.strftime('%Y%m%d')
current_date

'20241224'

In [26]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241224_PM_013629'

In [27]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [ ]:
# 수집한 파일 저장할 폴더 생성
    # 1:정치, 2:경제, 3:사회, 4:생활/문화, 5:IT/과학, 6:세계
if section_number == 1:
    section = 'politics'
    print('정치 섹션의 뉴스를 출력합니다.')
elif section_number == 2:
    section = 'economy'
    print('경제 섹션의 뉴스를 출력합니다.')
elif section_number == 3:
    section = 'society'
    print('사회 섹션의 뉴스를 출력합니다.')
elif section_number == 4:
    section = 'life_culture'
    print('생활/문화 섹션의 뉴스를 출력합니다.')
elif section_number == 5:
    section = 'it_science'
    print('IT/과학 섹션의 뉴스를 출력합니다.')
elif section_number == 6:
    section = 'world'
    print('세계 섹션의 뉴스를 출력합니다.')
    
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_news', section, current_date)
os.makedirs(crawled_folder_path, exist_ok=True)
print('파일 저장 위치 :', crawled_folder_path)

c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\crawled_data\naver_news\economy\20241224


In [29]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_news_{section}_{current_datetime}.xlsx'), index=False)

In [57]:
driver.quit()
print('크롤링을 종료합니다.')

크롤링을 종료합니다.


# END